In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [2]:
import sys
sys.path.append('/home/vtb70186744/dynbalance/')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# Дата из который мы прогнозируем (пока что не меняется)
train_end = datetime(year=2023, month=6, day=30)

# Горизонт прогноза в месяцах
horizon = 3

In [4]:
# Данные для прогноза
scenario_data = {
    # Ожидаемый баланс на первую дату прогноза, задавать необязательно
     'expected_amount':      [np.nan for h in range(horizon)],
    # ССВ
     'SSV':                  [0.48 for h in range(horizon)],
    # ФОР
     'FOR':                  [4.5 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [10.3 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [10 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [10 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [10 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [10 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [10 for h in range(horizon)],
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.1 for h in range(horizon)],
     'margin_[180d]':        [0.1 for h in range(horizon)],
     'margin_[365d]':        [0.1 for h in range(horizon)],
     'margin_[548d]':        [0.1 for h in range(horizon)],
     'margin_[730d]':        [0.2 for h in range(horizon)],
     'margin_[1095d]':       [0.2 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.4 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-1.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [11.2, 11.2, 11.2],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
scenario_df_user = pd.DataFrame(scenario_data)

In [5]:
scenario_df = preprocess_scenario(scenario_df_user, train_end, horizon)

In [6]:
port_folder = '/home/vtb70186744/dynbalance/data/portfolio_data'
portfolio = load_portfolio(train_end, port_folder)

In [7]:
# если хотим обучить модели

from core.models.utils import run_spark_session
#spark = run_spark_session('check_calc')

spark = None #если без обучения

In [8]:
folder = '/home/vtb70186744/dynbalance/data/trained_models'

sqlite_filepath = os.path.join(folder, 'modeldb_test.bin')

DB_URL = f"sqlite:///{sqlite_filepath}"
model_db = ModelDB(DB_URL)


In [9]:
ENV_NAME = 'hmelevskoi_env'

os.environ['CC'] = 'x86_64-conda-linux-gnu-gcc'
os.environ['CXX'] = 'x86_64-conda-linux-gnu-g++'
os.environ['PATH'] = os.path.abspath(f'/tmp/envs/{ENV_NAME}/bin') + ':' + os.environ['PATH']

In [10]:
config: ForecastConfig = ForecastConfig(
    first_train_end_dt = train_end,
    horizon = horizon,
    trainers = DepositModels.trainers,
    data_loaders = DepositModels.dataloaders,
    calculator_type = DepositIterativeCalculator,
    calc_type = DepositsCalculationType,
    adapter_types = DepositModels.adapter_types,
    scenario_data = scenario_df,
    portfolio = portfolio
)
    
training_manager = TrainingManager(spark, config.trainers, folder, model_db)   
engine: ForecastEngine = ForecastEngine(spark, config, training_manager, overwrite_models=False)

In [11]:
%%time
engine.run_all()

INFO:core:missing models: []
INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202306 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202306 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202306 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202306 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202306 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202306 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStruct

CPU times: user 39.1 s, sys: 4.71 s, total: 43.8 s
Wall time: 23.9 s


In [12]:
# вывод
portfolio_res = engine.calc_results['Deposits']['portfolio'] 
agg_res = engine.calc_results['Deposits']['agg_data']
maturity = engine.calc_results['Deposits']['maturity']
CurrentAccounts = engine.calc_results['CurrentAccounts']
SavingAccounts = engine.calc_results['SavingAccounts']
volumes = engine.calc_results['Volumes']

In [13]:
agg_res.groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,month_maturity,target_maturity_days,balance_start,balance_gain,balance,newbusiness,contract_close,early_withdrawal,operations,interests,renewal,universal_weight_id
report_dt,,,,,,,,,,,,,,
2023-07-31,32,27,1065,32373,3.148000e+12,-1.489729e+11,2.999027e+12,1.735497e+11,-2.428616e+11,-2.362218e+10,-7.262889e+10,1.659019e+10,4.980233e+10,304
2023-08-31,32,27,1065,32373,2.999027e+12,-1.335852e+11,2.865442e+12,1.667641e+11,-2.411727e+11,-2.166853e+10,-5.347260e+10,1.596451e+10,7.192395e+10,304
2023-09-30,34,27,1074,32643,2.865442e+12,-2.433125e+11,2.622129e+12,2.127669e+11,-4.108524e+11,-1.463416e+10,-4.503145e+10,1.443869e+10,1.044040e+11,309


In [14]:
# для записи и чтения экселя
# import pip
# pip.main(['install', 'openpyxl'])

In [15]:
# import openpyxl

In [16]:
# with pd.ExcelWriter("august_res_v2.xlsx") as writer:
#     portfolio_res.to_excel(writer, sheet_name='portfolio_res', index=False)
#     agg_res.to_excel(writer, sheet_name='agg_res', index=False)
#     maturity.to_excel(writer, sheet_name='maturity', index=False)
#     CurrentAccounts.to_excel(writer, sheet_name='CurrentAccounts', index=False)
#     SavingAccounts.to_excel(writer, sheet_name='SavingAccounts', index=False)
#     volumes.to_excel(writer, sheet_name='volumes', index=False)
#     pd.DataFrame(scenario_data).to_excel(writer, sheet_name='scenario', index=False)

## Считаем маржу

До момента прогноза используем данные с таблиц, далее - сценарные

In [17]:
import pip

In [18]:
pip.main(['install', 'openpyxl'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: http://nxrm:8081/repository/pypi/simple

Requirement already satisfied: openpyxl in /tmp/envs/hmelevskoi_env/lib/python3.7/site-packages (3.1.2)

Requirement already satisfied: et-xmlfile in /tmp/envs/hmelevskoi_env/lib/python3.7/site-packages (from openpyxl) (1.1.0)

0

## Импортируем расчет

In [19]:
from calc_margin_func import *

In [20]:
# portfolio_res = read_fact_data(4)
# portfolio_res_final = margin_calc(portfolio_res)

### Результаты на ФАКТИЧЕСКОМ портфеле

In [21]:
port_res = read_fact_data(7)

In [22]:
port_res_final = margin_calc(port_res)

In [23]:
port_res_final.groupby('report_dt').sum()['margin_value'] / 10**6

report_dt
2023-07-31    534.573166
Name: margin_value, dtype: float64

## Результаты для прогноза

In [24]:
portfolio_res = portfolio_res[portfolio_res['report_dt']=='2023-07-31']

In [25]:
portfolio_res_final =  margin_calc(portfolio_res)

In [26]:
portfolio_res_final.groupby('report_dt').sum()['margin_value'] / 10**6

report_dt
2023-07-31    483.653202
Name: margin_value, dtype: float64

In [27]:
# погрешность чпд
tmp1 = (port_res_final.groupby('report_dt').sum()['margin_value'] / 10**6).values[0]
tmp2 = (portfolio_res_final.groupby('report_dt').sum()['margin_value'] / 10**6).values[0]

print('погрешность чпд, %: ', round((abs(tmp1-tmp2)/tmp1)*100, 2))
print('погрешность чпд, млн: ', round(abs(tmp1-tmp2), 2))

погрешность чпд, %:  9.53
погрешность чпд, млн:  50.92


### Сравним факт и прогноз

Маржа считается:  
* FTP
    * дата открытия
* Weighted_rate
* баланс
    * в т.ч. по поколениям


ps - возможно где-то не сходится по процентам/едницам

### Общие параметры

#### Баланс

In [28]:
port_res['balance'].sum() / 10**9, portfolio_res['balance'].sum() / 10**9

(3150.01735808995, 2999.026997014009)

In [29]:
# погрешность прогноза портфеля
tmp3 = (port_res['balance'].sum() / 10**9)
tmp4 = (portfolio_res['balance'].sum() / 10**9)

print('погрешность портфеля, %: ', round((abs(tmp3-tmp4)/tmp3)*100, 2))
print('погрешность портфеля, млрд: ', round(abs(tmp3-tmp4), 2))

погрешность портфеля, %:  4.79
погрешность портфеля, млрд:  150.99


### weight_rate

In [30]:
(port_res['weight_rate'] * port_res['balance']).sum() / port_res['balance'].sum()

7.29388958931798

In [31]:
(portfolio_res['weight_rate'] * portfolio_res['balance']).sum() / portfolio_res['balance'].sum()

7.299215555042319

### Дополнить:

* дата прогноза как аргумент для расчета ftp ставок и спредов - нужно чтобы ограничить джойн, а после даты прогноза использовать сценарные ставки
* сценарий брать - чтобы корректно джойнить
* фтп ставки джойнить в портфеле - оттуда и брать средневшвешанные на момент открытия  
* спреды научиться брать с сайта для старых ретро-данных 

__Замечание:__ возможно часть не билось потому что фтп бралось с данных а не со сценария 